In [ ]:
import pandas as pd 
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset

TEST_DATA_PATH = "/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv"
MAX_LENGTH = 1024
MODEL_PATH = '/kaggle/input/es-deberta-large-fold0'
EVAL_BATCH_SIZE = 1

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

def tokenize(sample):
    return tokenizer(sample['full_text'], max_length=MAX_LENGTH, truncation=True)

df_test = pd.read_csv(TEST_DATA_PATH)
ds = Dataset.from_pandas(df_test).map(tokenize).remove_columns(['essay_id', 'full_text'])

model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
args = TrainingArguments(".", per_device_eval_batch_size=EVAL_BATCH_SIZE, report_to="none")
trainer = Trainer(model=model, args=args, data_collator=DataCollatorWithPadding(tokenizer), tokenizer=tokenizer)

df_test['score'] = trainer.predict(ds).predictions.argmax(-1) + 1
df_test.head()

In [ ]:
df_test[['essay_id', 'score']].to_csv('submission.csv', index=False)